# Introduction

We now have defined all of the necessary components to derive the equations of motion of the system using Kane's Method:

- Linear velocity of each mass center.
- Angular velocity of each rigid body.
- All contributing (external) forces.

We will use the `KanesMethod` class which provides an automated computation of the first order ordinary differential equations given the above quantities. (Any method can be used, such as Newton-Euler or Hamilton's method, but there are only automated methods for Kane's equations and Lagrangian dynamics at the moment).

# Setup

Import the solutions from the previous notebooks:

In [1]:
from __future__ import print_function, division
from solution.kinetics import *

We will need the `KanesMethod` class and `trigsimp` to give a reasonably compact result.

In [2]:
from sympy import trigsimp
from sympy.physics.mechanics import KanesMethod

Enable nice math printing:

In [3]:
from sympy.physics.vector import init_vprinting
init_vprinting(use_latex='mathjax')

In [4]:
KanesMethod?

# Equations of Motion

At the bare minimum for unconstrained systems, the `KanesMethod` class needs to know the generalized coordinates, the generalized speeds, the kinematical differential equations, the loads, the bodies, and a Newtonian reference frame. First, make a list of the generalized coordinates, i.e. the three joint angles:

In [5]:
coordinates = [theta1, theta2, theta3]
coordinates

[θ₁, θ₂, θ₃]

Now list the generalized speeds, i.e. the joint rates:

In [6]:
speeds = [omega1, omega2, omega3]
speeds

[ω₁, ω₂, ω₃]

The `KanesMethod` object can now be created with the inertial reference frame, coordinates, speeds, and kinematical differential equations:

In [7]:
kinematical_differential_equations

[ω₁ - θ₁̇, ω₂ - θ₂̇, ω₃ - θ₃̇]

In [8]:
kane = KanesMethod(inertial_frame, coordinates, speeds, kinematical_differential_equations)

Now list the six loads we defined and the three bodies:

In [9]:
loads = [lower_leg_grav_force,
         upper_leg_grav_force,
         torso_grav_force, 
         lower_leg_torque,
         upper_leg_torque,
         torso_torque]
loads

[(L_o, - g*m_L*I.y),
 (U_o, - g*m_U*I.y),
 (T_o, - g*m_T*I.y),
 (L, (T_a - T_k)*I.z),
 (U, (-T_h + T_k)*I.z),
 (T, T_h*I.z)]

In [10]:
bodies = [lower_leg, upper_leg, torso]
bodies

[Lower Leg, Upper Leg, Torso]

The equations of motion can now be computed using the `kanes_equations` method, which takes the list of loads and bodies. It returns the equations of motion (i.e. first order ordinary differential equations) in Kane's form:

$$ F_r + F_r^* = 0$$

which is essentially equivalent to the classic Newton_Euler form:

$$ \mathbf{F} = \mathbf{m}\mathbf{a} $$

$$ \mathbf{T} = \mathbf{I} \mathbf{\alpha} $$

In [11]:
fr, frstar = kane.kanes_equations(loads, bodies)

In [12]:
trigsimp(fr + frstar)

⎡                                                                            2
⎢d_L⋅g⋅m_L⋅sin(θ₁) + d_T⋅g⋅m_T⋅sin(θ₁ + θ₂ + θ₃) + d_T⋅l_L⋅m_T⋅(ω₁ + ω₂ + ω₃) 
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎣                                                                             

                              2                                     2         
⋅sin(θ₂ + θ₃) - d_T⋅l_L⋅m_T⋅ω₁ ⋅sin(θ₂ + θ₃) - d_T⋅l_U⋅m_T⋅(ω₁ + ω₂) ⋅sin(θ₃) 
                                                                              
                                                                              
                                                   

Keep in mind that out utlimate goal is to have the equations of motion in first order form:

$$ \dot{\mathbf{x}} = \mathbf{g}(\mathbf{x}, t) $$

The equations of motion are linear in terms of the derivatives of the generalized speeds and the `KanesMethod` class automatically puts the equations in a more useful form for the next step of numerical simulation:

$$ \mathbf{M}(\mathbf{x}, t)\dot{\mathbf{x}} = \mathbf{f}(\mathbf{x}, t) $$

Note that

$$ \mathbf{g} = \mathbf{M}^{-1}(\mathbf{x}, t) \mathbf{f}(\mathbf{x}, t) $$

and that $\mathbf{g}$ can be computed analytically but for non-toy problems, it is best to do this numerically. So we will simply generate the $\mathbf{M}$ and $\mathbf{f}$ matrices for later use.

The mass matrix, $\mathbf{M}$, can be accessed with the `mass_matrix` method (use `mass_matrix_full` to include the kinematical differential equations too. We can use `trigsimp` again to make this relatively compact: 

In [13]:
mass_matrix = trigsimp(kane.mass_matrix_full)
mass_matrix

⎡1  0  0                                                                      
⎢                                                                             
⎢0  1  0                                                                      
⎢                                                                             
⎢0  0  1                                                                      
⎢                                                                             
⎢                                 2           ⎛   2                           
⎢0  0  0  I_Lz + I_Tz + I_Uz + d_L ⋅m_L + m_T⋅⎝d_T  + 2⋅d_T⋅l_L⋅cos(θ₂ + θ₃) +
⎢                                                                             
⎢                                                                            ⎛
⎢0  0  0                     I_Tz + I_Uz + d_U⋅m_U⋅(d_U + l_L⋅cos(θ₂)) + m_T⋅⎝
⎢                                                                             
⎢                                                   

The right hand side, $\mathbf{f}$, is a vector function of all the non-inertial forces (gyroscopic, external, coriolis, etc):

In [14]:
forcing_vector = trigsimp(kane.forcing_full)
forcing_vector

⎡                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                            2
⎢d_L⋅g⋅m_L⋅sin(θ₁) + d_T⋅g⋅m_T⋅sin(θ₁ + θ₂ + θ₃) + d_T⋅l_L⋅m_T⋅(ω₁ + ω₂ + ω₃) 
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                   